In [2]:
# Description: Face Recognition using OpenCV and Python

import cv2
import os

# Defining paths

SRC_PATH = os.getcwd()
ASSETS_PATH = os.path.join(SRC_PATH, 'assets')
IMAGES_PATH = os.path.join(ASSETS_PATH, 'Images')

Loading and preprocessing the image

In [3]:
from cv2 import Mat

def show_image(image_path:str = None, cv2_image:Mat = None, title:str="Image") -> None:
    """Shows an image in a window with cv2. And waits for a key to be pressed to close the window.
    The path to the image or the image object must be provided.

    Args:
        image_path (str, optional): Path to the image to show. Defaults to None.
        cv2_image (Mat, optional): Image object to show. Defaults to None.
        title (str, optional): Title of the window containing the shown image. Defaults to "Image".
    """
    if image_path is None and cv2_image is None:
        raise Exception("No image to show")
    
    if image_path is not None:
        image_to_show = cv2.imread(image_path)
    else: 
        image_to_show = cv2_image
    
    cv2.imshow(title, image_to_show)
    cv2.waitKey(0)
    cv2.destroyWindow(title)

In [4]:
people_image = os.path.join(IMAGES_PATH, "people1.jpg")
image = cv2.imread(people_image)
image.shape # (height, width, channels), channels are BGR, has three channels. Tougher to process.

(1280, 1920, 3)

In [5]:
show_image(image_path=people_image, title="People Image") # Shows the image in a window.

In [6]:
image = cv2.resize(image, (800, 600)) # Resizes the image to 800x600 pixels.
show_image(cv2_image=image, title="Resized Image")
image.shape # (600, 800, 3). Still has three channels. Must be converted to grayscale.

(600, 800, 3)

In [7]:
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # Converts the image to grayscale.
show_image(cv2_image=gray_image, title="Gray Image")
image.shape # (600, 800). Now has only one channel.

(600, 800, 3)

Face detection

In [8]:
facial_detector_path = os.path.join(ASSETS_PATH, "Cascades", "haarcascade_frontalface_default.xml")
facial_detector = cv2.CascadeClassifier(facial_detector_path)
detections = facial_detector.detectMultiScale(gray_image, scaleFactor=1.09) # Detects faces in the image.
print("Number of faces detected: ", len(detections))
detections # Array of tuples. Each tuple contains the coordinates of the top left corner of the face 
           # and the width and height of the face.

Number of faces detected:  5


array([[386, 232,  74,  74],
       [ 90, 238,  69,  69],
       [113, 122,  56,  56],
       [475, 122,  60,  60],
       [678,  74,  65,  65]])

In [9]:
def draw_rectangles_in_image(image:Mat, rectangles:list[tuple], color:tuple=(0, 255, 0), thickness:int = 2) -> None:
    """Draws green rectangles in an image.

    Args:
        image (Mat): Image to draw the rectangles in.
        rectangles (list): List of tuples. Each tuple contains the coordinates of the top left corner of the rectangle 
                           and the width and height of the rectangle.
        color (tuple, optional): Color of the rectangles. Defaults to (0, 255, 0), green.
        thickness (int, optional): Thickness of the rectangles. Defaults to 2.
    """
    for (x, y, w, h) in rectangles:
        cv2.rectangle(image, (x, y), (x+w, y+h), color, thickness) # (x, y) is the top left corner of the rectangle.
                                                                 # (x+w, y+h) is the bottom right corner of the rectangle.

In [10]:
draw_rectangles_in_image(image, detections) # Draws detections in the image as green rectangles.
show_image(cv2_image=image, title="Detections")

In [11]:
# Detecting faces in a second image

# Loading and preparing the second image
image_2_path = os.path.join(IMAGES_PATH, "people2.jpg")
image_2 = cv2.imread(image_2_path)
gray_image_2 = cv2.cvtColor(image_2, cv2.COLOR_BGR2GRAY)

# Detecting faces in the second image
detections_2 = facial_detector.detectMultiScale(gray_image_2, scaleFactor=1.2, minNeighbors=3,
                                             minSize=(32,32), maxSize=(100,100)) # Even through ajusting the parameters, the F1 score isn't 100%.

# Drawing the detections in the second image
draw_rectangles_in_image(image_2, detections_2)

# Showing the second image with the detections
show_image(cv2_image=image_2, title="Detections 2")

In [15]:
# Eye detection in first image, which had to be resized, because the eyes are smaller than the faces.
image = cv2.resize(image, (1920, 1280))
gray_image = cv2.resize(gray_image, (1920, 1280))

# Detecting eyes in the first image
eye_detector_path = os.path.join(ASSETS_PATH, "Cascades", "haarcascade_eye.xml")
eye_detector = cv2.CascadeClassifier(eye_detector_path)
eye_detections = eye_detector.detectMultiScale(gray_image, scaleFactor=1.24, minNeighbors=10, maxSize=(70,70))

# Drawing the eye detections in the first image
draw_rectangles_in_image(image, eye_detections, color=(255, 0, 0))

# Showing the first image with the eye detections
show_image(cv2_image=image, title="Eye Detections")